In [ ]:
#JUAN DIEGO POZO ORTIZ 
# CÓDIGO: 20210679


In [5]:
from  fiona import listlayers
import os

thailandMaps=os.path.join('maps','thailandMaps_4239.gpkg')

#layers in maps
listlayers(thailandMaps)

['country', 'cities', 'rivers']

In [8]:
import os
import geopandas as gpd

cities =gpd.read_file(thailandMaps,layer='cities')


rivers=gpd.read_file(thailandMaps,layer='rivers')
